# Section 2.5: Introduction to Measure Theory - Exercises

This notebook contains practical exercises related to measure theory concepts and their applications to complex systems. We'll explore how to implement and visualize key measure theory concepts using Python.

## Learning Objectives
- Implement computations related to measure spaces and sigma-algebras
- Work with measure functions and their properties in Python
- Approximate Lebesgue integration numerically
- Explore the connections between measure theory and probability theory
- Apply measure-theoretic concepts to analyze complex systems

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from matplotlib.patches import Rectangle
from itertools import chain, combinations
import seaborn as sns
from collections import defaultdict
import pandas as pd

# Set plotting style
plt.style.use('seaborn-whitegrid')
sns.set_context("notebook", font_scale=1.5)

## Exercise 1: Visualizing Sigma-Algebras and Measure Spaces (Fully Solved)

In this exercise, we'll work with a finite set and explore the concept of sigma-algebras through visualization and computation. This will help us understand how measure spaces are constructed and how measures assign values to sets.

### Problem Statement:
Consider a simple finite set Ω = {1, 2, 3, 4}.

1. Generate the power set of Ω (which forms a sigma-algebra)
2. Create a subset of the power set that still satisfies the sigma-algebra properties
3. Define a measure function on both sigma-algebras
4. Visualize the sets and their measures
5. Verify the key measure properties: non-negativity, empty set measure = 0, and countable additivity

Let's start by generating the power set of Ω.

In [ ]:
# Function to generate power set (all possible subsets) of a set
def power_set(s):
    """Return the power set of set s"""
    return list(map(set, chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))))

# Our sample space
omega = {1, 2, 3, 4}

# Generate the power set (which is a sigma-algebra)
power_set_omega = power_set(omega)

# Print all elements in the power set (which is our largest sigma-algebra)
print("Power set of Ω = {1, 2, 3, 4} contains the following sets:")
for i, subset in enumerate(power_set_omega):
    print(f"{i+1}. {subset}")
    
print(f"\nThe power set contains {len(power_set_omega)} elements.")

Now, let's create a smaller sigma-algebra on the same set Ω. A valid sigma-algebra must contain:
1. The empty set ∅
2. The entire sample space Ω
3. Be closed under complement
4. Be closed under countable unions

Let's create a simple sigma-algebra containing ∅, Ω, {1, 2}, and {3, 4}.

In [ ]:
# Define a smaller sigma-algebra
smaller_sigma_algebra = [set(), {1, 2, 3, 4}, {1, 2}, {3, 4}]

# Function to verify if a collection of sets is a valid sigma-algebra
def is_sigma_algebra(collection, universe):
    """Check if a collection of sets forms a sigma-algebra"""
    # Check if empty set is included
    if set() not in collection:
        return False, "Empty set is missing"
    
    # Check if universe is included
    if set(universe) not in collection:
        return False, "Universe set is missing"
    
    # Check closure under complement
    for s in collection:
        complement = set(universe) - s
        if complement not in collection:
            return False, f"Complement of {s} = {complement} is missing"
    
    # Check closure under finite unions (for simplicity, we'll check pairs)
    for s1 in collection:
        for s2 in collection:
            union = s1.union(s2)
            if union not in collection:
                return False, f"Union of {s1} and {s2} = {union} is missing"
    
    return True, "Valid sigma-algebra"

# Verify our smaller collection is a valid sigma-algebra
is_valid, message = is_sigma_algebra(smaller_sigma_algebra, omega)
print(f"Is our smaller collection a valid sigma-algebra? {is_valid}")
print(f"Message: {message}")

Now, let's define a measure function on our sigma-algebras. A measure must satisfy:
1. Non-negativity: μ(A) ≥ 0 for all sets A
2. Null empty set: μ(∅) = 0
3. Countable additivity: For disjoint sets, the measure of their union equals the sum of their measures

For our example, we'll create two different measures:

In [ ]:
# Define a measure on the power set (counting measure - counts the elements in each set)
def counting_measure(s):
    return len(s)

# Define a weighted measure that assigns different weights to different elements
weights = {1: 0.1, 2: 0.2, 3: 0.3, 4: 0.4}
def weighted_measure(s):
    return sum(weights.get(item, 0) for item in s)

# Calculate measures for both sigma-algebras
power_set_measures = {frozenset(s): counting_measure(s) for s in power_set_omega}
smaller_sigma_measures = {frozenset(s): weighted_measure(s) for s in smaller_sigma_algebra}

# Display the measures
print("Counting Measure on Power Set:")
for s, measure in power_set_measures.items():
    print(f"μ({set(s)}) = {measure}")

print("\nWeighted Measure on Smaller Sigma-Algebra:")
for s, measure in smaller_sigma_measures.items():
    print(f"μ({set(s)}) = {measure}")

Now, let's visualize these measure spaces using a heatmap representation:

In [ ]:
def visualize_measure_space(sets, measure_func, title):
    """Visualize a measure space with a heatmap representation"""
    # Create a matrix representation of set inclusion and measures
    elements = list(range(1, 5))  # Our universe {1,2,3,4}
    set_list = sorted(sets, key=len)  # Sort sets by size for better visualization
    
    # Create the matrix: rows=sets, columns=elements
    matrix = np.zeros((len(set_list), len(elements)))
    for i, s in enumerate(set_list):
        for j, elem in enumerate(elements):
            if elem in s:
                matrix[i, j] = 1
    
    # Calculate measures
    measures = [measure_func(s) for s in set_list]
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, len(set_list) * 0.5 + 2), 
                                   gridspec_kw={'width_ratios': [3, 1]})
    
    # Plot set membership matrix
    sns.heatmap(matrix, cmap="Blues", cbar=False, ax=ax1, 
                xticklabels=elements,
                yticklabels=[str(s) for s in set_list])
    ax1.set_title("Set Membership")
    ax1.set_xlabel("Elements")
    ax1.set_ylabel("Sets")
    
    # Plot measures
    measure_matrix = np.array(measures).reshape(-1, 1)
    sns.heatmap(measure_matrix, cmap="Reds", cbar=True, ax=ax2,
                xticklabels=['Measure'],
                yticklabels=[str(s) for s in set_list])
    ax2.set_title("Measure Values")
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Visualize both measure spaces
visualize_measure_space(power_set_omega, counting_measure, "Counting Measure on Power Set")
visualize_measure_space(smaller_sigma_algebra, weighted_measure, "Weighted Measure on Smaller Sigma-Algebra")

Let's verify the key properties of a measure function for our weighted measure on the smaller sigma-algebra:

In [ ]:
# 1. Non-negativity: μ(A) ≥ 0 for all sets A
all_non_negative = all(weighted_measure(s) >= 0 for s in smaller_sigma_algebra)
print(f"1. Non-negativity satisfied: {all_non_negative}")

# 2. Null empty set: μ(∅) = 0
empty_set_measure = weighted_measure(set())
print(f"2. Null empty set: μ(∅) = {empty_set_measure}")

# 3. Countable additivity for disjoint sets
# In our smaller sigma-algebra, {1,2} and {3,4} are disjoint
set1 = {1, 2}
set2 = {3, 4}
union_measure = weighted_measure(set1.union(set2))
sum_measures = weighted_measure(set1) + weighted_measure(set2)

print(f"3. Countable additivity:")
print(f"   μ({set1} ∪ {set2}) = {union_measure}")
print(f"   μ({set1}) + μ({set2}) = {weighted_measure(set1)} + {weighted_measure(set2)} = {sum_measures}")
print(f"   Are they equal? {np.isclose(union_measure, sum_measures)}")

### Application to Complex Systems: Fractal Dimension Calculation

One important application of measure theory in complex systems is the calculation of fractal dimensions. Let's implement the box-counting method to estimate the fractal dimension of a simple fractal - the Sierpinski triangle:

In [ ]:
def sierpinski_points(iterations, points=None):
    """Generate points in a Sierpinski triangle"""
    if points is None:
        # Initial triangle vertices
        points = np.array([[0, 0], [0.5, np.sqrt(3)/2], [1, 0]])
        
    if iterations == 0:
        return points
    
    # Generate new points for the next iteration
    new_points = np.zeros((3 * len(points), 2))
    for i, p in enumerate(points):
        # Three transformations for the Sierpinski triangle
        new_points[3*i] = p / 2  # Bottom left
        new_points[3*i+1] = p / 2 + np.array([0.25, np.sqrt(3)/4])  # Top
        new_points[3*i+2] = p / 2 + np.array([0.5, 0])  # Bottom right
    
    return sierpinski_points(iterations - 1, new_points)

# Generate Sierpinski triangle points with 5 iterations
sierpinski = sierpinski_points(5)

# Plot the fractal
plt.figure(figsize=(10, 8))
plt.scatter(sierpinski[:, 0], sierpinski[:, 1], s=0.5, c='black')
plt.title('Sierpinski Triangle')
plt.axis('equal')
plt.axis('off')
plt.show()

# Perform box counting to estimate fractal dimension
def box_counting(points, box_sizes):
    """Estimate fractal dimension using box counting method"""
    # Normalize points to [0, 1] range
    min_vals = np.min(points, axis=0)
    max_vals = np.max(points, axis=0)
    norm_points = (points - min_vals) / (max_vals - min_vals)
    
    # Count boxes for different box sizes
    box_counts = []
    for box_size in box_sizes:
        # Create grid and count occupied boxes
        grid_size = int(1/box_size)
        occupied_boxes = set()
        
        for p in norm_points:
            # Calculate box indices
            box_x = int(p[0] * grid_size)
            box_y = int(p[1] * grid_size)
            occupied_boxes.add((box_x, box_y))
        
        box_counts.append(len(occupied_boxes))
    
    return box_counts

# Box sizes for calculation
box_sizes = np.logspace(-3, -1, 10)  # From 0.001 to 0.1
counts = box_counting(sierpinski, box_sizes)

# Plot log-log relationship to estimate fractal dimension
plt.figure(figsize=(10, 6))
plt.loglog(1/box_sizes, counts, 'o-')
plt.xlabel('1/Box Size')
plt.ylabel('Number of Boxes')
plt.title('Box Counting Method for Sierpinski Triangle')
plt.grid(True, which="both", ls="-")

# Calculate fractal dimension (slope of log-log plot)
log_sizes = np.log(1/box_sizes)
log_counts = np.log(counts)
slope, intercept = np.polyfit(log_sizes, log_counts, 1)

# Add the fitted line
plt.plot(1/box_sizes, np.exp(intercept) * (1/box_sizes)**slope, 'r-', 
         label=f'Estimated Dimension = {slope:.4f}')
plt.legend()
plt.show()

print(f"Estimated fractal dimension: {slope:.4f}")
print(f"Theoretical fractal dimension for Sierpinski triangle: log(3)/log(2) = {np.log(3)/np.log(2):.4f}")

### Conclusion of Exercise 1

In this exercise, we've explored fundamental concepts of measure theory:

1. We constructed and visualized sigma-algebras on a finite set
2. We defined and verified different measure functions
3. We confirmed the key properties that all measures must satisfy
4. We applied measure theory to calculate the fractal dimension of a Sierpinski triangle

This demonstrates how measure theory provides a mathematical framework for quantifying complex structures like fractals, which are common in complex systems. The box-counting dimension is just one example of a measure-theoretic approach to characterizing complexity.

## Exercise 2: Lebesgue Integration and Applications to Probability Theory (For You to Solve)

In this exercise, you'll explore Lebesgue integration and its applications to probability theory and complex systems. The Lebesgue integral extends the concept of integration to a wider class of functions than the Riemann integral and is fundamental to modern probability theory.

### Problem Statement:

Consider a complex system with a state variable that follows a heavy-tailed distribution. Specifically, the probability density function (PDF) is given by:

$$f(x) = \frac{\alpha}{x^{\alpha+1}}$$

for $x \geq 1$ and $\alpha > 0$ (this is a Pareto distribution, often used to model systems exhibiting power-law behavior).

Your tasks:

1. Implement a numerical approximation of the Lebesgue integral to calculate probabilities
2. Compare it with standard numerical integration (Riemann integral)
3. Calculate and visualize the cumulative distribution function (CDF)
4. Investigate how the parameter $\alpha$ affects the moments of the distribution
5. Apply this to a simulated complex system with power-law behavior

### Getting Started:

Here's a template to begin your solution:

In [ ]:
# Function for the Pareto PDF
def pareto_pdf(x, alpha):
    """Pareto probability density function"""
    if isinstance(x, (list, np.ndarray)):
        result = np.zeros_like(x, dtype=float)
        valid = x >= 1
        result[valid] = alpha / (x[valid]**(alpha+1))
        return result
    else:
        return alpha / (x**(alpha+1)) if x >= 1 else 0

# Function for the Pareto CDF (theoretical)
def pareto_cdf(x, alpha):
    """Pareto cumulative distribution function"""
    if isinstance(x, (list, np.ndarray)):
        result = np.zeros_like(x, dtype=float)
        valid = x >= 1
        result[valid] = 1 - (1/x[valid])**alpha
        return result
    else:
        return 1 - (1/x)**alpha if x >= 1 else 0

# TODO: Implement a numerical approximation of the Lebesgue integral
def lebesgue_integral_approx(f, domain, levels):
    """Approximate the Lebesgue integral of function f over domain
    using level sets approach"""
    # Your implementation here
    pass

# TODO: Calculate CDF using both Lebesgue approximation and Riemann integration
# Compare the results

# TODO: Investigate how alpha affects the moments of the distribution
# Expected value, variance, and higher moments

# TODO: Apply to a simulated complex system with power-law behavior
# For example, a network with degree distribution following a power law

### Hints for Your Solution:

1. For the Lebesgue integral approximation, consider using a level set approach:
   - Partition the range of the function into levels
   - For each level, measure the set where the function exceeds that level
   - The Lebesgue integral is approximated as the sum of these measures multiplied by the level increments

2. For the Riemann integral approximation, you can use `scipy.integrate.quad`

3. For the analysis of moments, note that:
   - The $k$-th moment $E[X^k]$ of a Pareto distribution exists only if $k < \alpha$
   - This is a characteristic property of heavy-tailed distributions

4. For the complex systems application, consider:
   - Generating a network with a power-law degree distribution
   - Analyzing how the network's properties depend on the parameter $\alpha$

Good luck! This exercise will deepen your understanding of measure theory, Lebesgue integration, and their applications to complex systems with heavy-tailed distributions.